In [2]:
import pandas as pd
import numpy as np
import time
import os

In [3]:

movie_all_info_path = 'data/m_all_info.csv'
movie_all_column_names = ['movie_id', 'title', 'director', 'scriptwriter', 'actors', 'genres',
                          'place', 'languages', 'time', 'duration', 'other_names', '_']
user_movie_score_path = 'data/u_score.csv' 
user_movie_column_names = ['movie_id', 'user_id', 'rating']

ITEM_ITEM_SIMMAT_PATH = 'data/ITEM_ITEM_SIMMAT.csv'

In [4]:
ITEM_ITEM_SIMMAT = pd.read_csv(ITEM_ITEM_SIMMAT_PATH, index_col=0)
ITEM_ITEM_SIMMAT.index

Int64Index([30163509,  4840388, 26369709, 30287729, 30349667, 27622447,
            25986662, 26374197, 30331149, 27109679,
            ...
             1291818,  1291828, 26389601, 25798222,  1305164,  1966460,
             3070921,  1299436,  1294540, 27126359],
           dtype='int64', name='movie_id', length=1663)

In [9]:
def init_movie_data():
    """加载所有电影数据"""
    data = pd.read_csv(movie_all_info_path, names=movie_all_column_names,
        sep=r',\s*', engine='python', encoding='utf-8')
    return data


def init_rating_data():
    """加载电影评分数据"""
    data = pd.read_csv(user_movie_score_path, names=user_movie_column_names, usecols=(0, 1, 2))
    return data


def init_user_item_mat():
    """
    初始化User对Item的评分矩阵。并全局存储。
    """
    # 获取不重复的user_id列表与movie_id列表作为矩阵行索引标签与列索引标签
    unique_movieids = rating_data['movie_id'].unique()
    unique_userids = rating_data['user_id'].unique()
    user_item_mat = pd.DataFrame(
        0, index=unique_userids, columns=unique_movieids)
    # 遍历rating_data中的每一行数据，充实评分矩阵的内容
    for index, row in rating_data.iterrows():
        user_item_mat[row['movie_id']][row['user_id']] = row['rating']
    return user_item_mat


In [10]:
movie_data = init_movie_data()
rating_data = init_rating_data()
user_item_mat = init_user_item_mat()


In [14]:
movie_data

,movie_id,title,director,scriptwriter,actors,genres,place,languages,time,duration,other_names,_
0,30163509,飞驰人生,韩寒,韩寒,沈腾 / 黄景瑜 / 尹正 / 张本煜 / 尹昉 / 田雨 / 魏翔 / 赵文瑄 / 腾格尔...,剧情 / 励志 / 喜剧,中国大陆,汉语普通话 / 英语,2019-02-05(中国大陆),98分钟,Pegasus,tt9597190
1,4840388,新喜剧之王,周星驰,周星驰,王宝强 / 鄂靖文 / 张全蛋 / 景如洋 / 张琪 / 袁兴哲 / 田启文 / 黄骁鹏 /...,剧情 / 喜剧,中国大陆 / 香港,汉语普通话 / 粤语,2019-02-05(中国大陆),91分钟,喜剧之王2 / D计划 / The New King of Comedy / King of...,tt9368628
2,26369709,欢迎来到马文镇 Welcome to Marwen,罗伯特·泽米吉斯,罗伯特·泽米吉斯 / 卡罗琳·汤普森,史蒂夫·卡瑞尔 / 法尔克·亨特切尔 / 马特·欧莱瑞 / 尼古莱·维切尔 / 帕特里克·罗...,剧情 / 传记,美国,英语,2018-12-21(美国),116分钟,马克的异乡世界(台) / 马文科尔 / Marwencol / The Women of M...,tt3289724
3,30287729,正义联盟大战致命五人组 Justice League vs. The Fatal Five,刘山姆,Jim Shooter,凯文·康瑞 / 乔治·纽伯恩 / 苏珊·爱森伯格 / 丹妮拉·玻芭迪拉 / 素玛立·蒙塔诺 ...,动画,美国,英语,2019-03-30(美国),Nan,Nan,tt8752474
4,30349667,指挥家 De dirigent,玛利亚·彼特斯,玛利亚·彼特斯,克里斯蒂亚娜 ‧德‧布奥恩 / 本杰明·温赖特 / 斯科特·特纳·菲尔德 / 塞马斯·F·萨...,传记,荷兰,英语,2018-10-25(荷兰),137分钟,The Conductor,tt6932818
...,...,...,...,...,...,...,...,...,...,...,...,...
1981,26661189,脱单告急,柯孟融,任鹏,董子健 / 钟楚曦 / 春夏 / 袁福福 / 靳锦 / 尹雨航 / 吴昱瑶 / 刘梦梦 / 郗婧妍,喜剧 / 爱情,中国大陆,汉语普通话,2018-04-20(中国大陆) / 2018-04-15(北京国际电影节),106分钟,完全男生手册 / Dude's Manual,tt8319694
1982,27145025,精灵宝可梦：大家的故事 劇場版 ポケットモンスター みんなの物語,矢岛哲生,梅原英司 / 高羽彩,松本梨香 / 大谷育江 / 林原惠美 / 三木真一郎 / 犬山犬子 / 石冢运升 / 川荣李...,动画 / 奇幻 / 冒险,日本,日语,2018-07-13(日本),97分钟,精灵宝可梦剧场版：大家的故事 / 剧场版精灵宝可梦：我们的故事(港) / 剧场版口袋妖怪 大...,tt8108230
1983,27107625,代号基亚斯：反叛的鲁路修3之皇道 コードギアス 反逆のルルーシュⅢ 皇道,谷口悟朗,大河内一楼 / 谷口悟朗,福山润 / 樱井孝宏 / 野上尤加奈 / 小清水亚美 / 名塚佳织 / 绿川光 / 金井美香...,动作 / 科幻 / 动画,日本,日语,2018-05-26(日本),140分钟,Nan,Nan
1984,27611038,猫是要抱着的 猫は抱くもの,犬童一心,高田亮 / 大山淳子,泽尻英龙华 / 吉泽亮 / 峯田和伸 / 岩松了,剧情,日本,日语,2018-06-18(上海电影节) / 2018-06-23(日本),110分钟,猫是用来抱的(台) / 拥抱猫咪 / The Cat In Their Arms,tt9742106


In [6]:
user_item_mat.head()

,26411377,30267308,30181455,26776469,1292271,6973376,11502153,3771562,3319755,1297052,...,1291818,1291828,26389601,25798222,1305164,1966460,3070921,1299436,1294540,27126359
165852925,4,3,2,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,0,0,0,0,5,5,3,5,5,5,...,0,0,0,0,0,0,0,0,0,0
130012755,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
150932792,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
189793987,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
itemids = user_item_mat.columns
ITEM_ITEM_SIMMAT = pd.DataFrame(0.0, index=itemids, columns=itemids, dtype='f8')

In [8]:
movie_feature_map = pd.get_dummies(movie_data, columns=['director']) 
movie_feature_map.head(2)

,movie_id,title,scriptwriter,actors,genres,place,languages,time,duration,other_names,...,director_黄泰来,director_黄渤,director_黄瀬和哉,director_黄荣昇,director_黄进,director_黎继强,director_黑泽明,director_黑泽清,director_黛布拉·格兰尼克,director_민철기 / 노시용
0,30163509,飞驰人生,韩寒,沈腾 / 黄景瑜 / 尹正 / 张本煜 / 尹昉 / 田雨 / 魏翔 / 赵文瑄 / 腾格尔...,剧情 / 励志 / 喜剧,中国大陆,汉语普通话 / 英语,2019-02-05(中国大陆),98分钟,Pegasus,...,0,0,0,0,0,0,0,0,0,0
1,4840388,新喜剧之王,周星驰,王宝强 / 鄂靖文 / 张全蛋 / 景如洋 / 张琪 / 袁兴哲 / 田启文 / 黄骁鹏 /...,剧情 / 喜剧,中国大陆 / 香港,汉语普通话 / 粤语,2019-02-05(中国大陆),91分钟,喜剧之王2 / D计划 / The New King of Comedy / King of...,...,0,0,0,0,0,0,0,0,0,0


In [9]:
# 整理导演字段多标签二值化处理
L = pd.DataFrame(movie_data['actors'].str.split(' / ', expand=True)).fillna('').values
import sklearn.preprocessing as sp
mlb = sp.MultiLabelBinarizer()
res = pd.DataFrame(mlb.fit_transform(L), columns=mlb.classes_)
movie_feature_map2 = pd.concat([movie_feature_map, res], axis=1)
print(movie_feature_map2.shape)

(1986, 15881)


In [10]:
# 整理电影类型字段多标签二值化处理
G = pd.DataFrame(movie_data['genres'].str.split(' / ', expand=True)).fillna('').values
import sklearn.preprocessing as sp
mlb = sp.MultiLabelBinarizer()
res = pd.DataFrame(mlb.fit_transform(G), columns=mlb.classes_)
movie_feature_map3 = pd.concat([movie_feature_map2, res], axis=1)
print(movie_feature_map3.shape)
movie_feature_map3.head(1)

(1986, 15915)


,movie_id,title,scriptwriter,actors,genres,place,languages,time,duration,other_names,...,犯罪,真人秀,短片,科幻,纪录片,脱口秀,西部,运动,音乐,黑色电影
0,30163509,飞驰人生,韩寒,沈腾 / 黄景瑜 / 尹正 / 张本煜 / 尹昉 / 田雨 / 魏翔 / 赵文瑄 / 腾格尔...,剧情 / 励志 / 喜剧,中国大陆,汉语普通话 / 英语,2019-02-05(中国大陆),98分钟,Pegasus,...,0,0,0,0,0,0,0,0,0,0


In [11]:
movie_feature_map3.drop(movie_feature_map3.columns[np.arange(len(movie_all_column_names))], axis=1, inplace=True)
movie_feature_map3.T


,0,1,2,3,4,5,6,7,8,9,...,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985
director_Antonio Carluccio,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
director_Anu Menon,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
director_Athanassios Vakalis,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
director_Babis Makridis,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
director_Charles Haid / Chris Long,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
脱口秀,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
西部,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
运动,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
音乐,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:

MOVIE_DATA_PATH = 'data/m_all_info.csv'
MOVIE_DATA_COLUMNS = ['movie_id', 'title', 'director', 'scriptwriter', 'actors', 'genres',
                          'place', 'languages', 'time', 'duration', 'other_names', '_']
RATING_DATA_PATH = 'data/u_score.csv' 
RATING_DATA_CLOLUMNS = ['movie_id', 'user_id', 'rating']

ITEM_ITEM_SIMMAT_PATH = 'data/ITEM_ITEM_SIMMAT.csv'

def init_movie_data():
    """加载所有电影数据"""
    data = pd.read_csv(MOVIE_DATA_PATH, names=MOVIE_DATA_COLUMNS,
        sep=r',\s*', engine='python', encoding='utf-8')
    data.drop_duplicates(subset=['movie_id'], keep='first', inplace=True)
    print('Movie Data loaded:', data.shape)
    return data

def init_rating_data():
    """加载电影评分数据"""
    data = pd.read_csv(RATING_DATA_PATH, names=RATING_DATA_CLOLUMNS, usecols=(0, 1, 2))
    print('Rating Data loaded:', data.shape)
    return data


def init_user_item_mat():
    """
    初始化User对Item的评分矩阵。并全局存储。
    """
    # 获取不重复的user_id列表与movie_id列表作为矩阵行索引标签与列索引标签
    unique_movieids = rating_data['movie_id'].unique()
    unique_userids = rating_data['user_id'].unique()
    user_item_mat = pd.DataFrame(
        0, index=unique_userids, columns=unique_movieids)
    # 遍历rating_data中的每一行数据，充实评分矩阵的内容
    for index, row in rating_data.iterrows():
        user_item_mat[row['movie_id']][row['user_id']] = row['rating']
    print('User_item_mat loaded:', user_item_mat.shape)
    return user_item_mat

movie_data = init_movie_data()
rating_data = init_rating_data()
user_item_mat = init_user_item_mat()
ITEM_ITEM_SIMMAT = None
if os.path.exists(ITEM_ITEM_SIMMAT_PATH):
    print('load model ITEM_ITEM_SIMMAT')
    ITEM_ITEM_SIMMAT = pd.read_csv(ITEM_ITEM_SIMMAT_PATH, index_col=0, header=0)


movie_data.index = movie_data['movie_id']
movie_feature_map = pd.get_dummies(movie_data, columns=['director']) 
print(movie_feature_map.shape)
movie_feature_map.index

Movie Data loaded: (1663, 12)
Rating Data loaded: (4053, 3)
User_item_mat loaded: (401, 1551)
(1663, 1238)


Int64Index([30163509,  4840388, 26369709, 30287729, 30349667, 27622447,
            25986662, 26374197, 30331149, 27109679,
            ...
             1291818,  1291828, 26389601, 25798222,  1305164,  1966460,
             3070921,  1299436,  1294540, 27126359],
           dtype='int64', name='movie_id', length=1663)

In [35]:
L = pd.DataFrame(movie_data['actors'].str.split(' / ', expand=True)).fillna('').values
import sklearn.preprocessing as sp
mlb = sp.MultiLabelBinarizer()
res = pd.DataFrame(mlb.fit_transform(L), columns=mlb.classes_, index=movie_data['movie_id'])
movie_feature_map2 = pd.concat([movie_feature_map, res], axis=1)
movie_feature_map2

,movie_id,title,scriptwriter,actors,genres,place,languages,time,duration,other_names,...,미나,미리아,민영,사나,소미,정연,정원,지효,쯔위,채영
movie_id,,,,,,,,,,,,,,,,,,,,,
30163509,30163509,飞驰人生,韩寒,沈腾 / 黄景瑜 / 尹正 / 张本煜 / 尹昉 / 田雨 / 魏翔 / 赵文瑄 / 腾格尔...,剧情 / 励志 / 喜剧,中国大陆,汉语普通话 / 英语,2019-02-05(中国大陆),98分钟,Pegasus,...,0,0,0,0,0,0,0,0,0,0
4840388,4840388,新喜剧之王,周星驰,王宝强 / 鄂靖文 / 张全蛋 / 景如洋 / 张琪 / 袁兴哲 / 田启文 / 黄骁鹏 /...,剧情 / 喜剧,中国大陆 / 香港,汉语普通话 / 粤语,2019-02-05(中国大陆),91分钟,喜剧之王2 / D计划 / The New King of Comedy / King of...,...,0,0,0,0,0,0,0,0,0,0
26369709,26369709,欢迎来到马文镇 Welcome to Marwen,罗伯特·泽米吉斯 / 卡罗琳·汤普森,史蒂夫·卡瑞尔 / 法尔克·亨特切尔 / 马特·欧莱瑞 / 尼古莱·维切尔 / 帕特里克·罗...,剧情 / 传记,美国,英语,2018-12-21(美国),116分钟,马克的异乡世界(台) / 马文科尔 / Marwencol / The Women of M...,...,0,0,0,0,0,0,0,0,0,0
30287729,30287729,正义联盟大战致命五人组 Justice League vs. The Fatal Five,Jim Shooter,凯文·康瑞 / 乔治·纽伯恩 / 苏珊·爱森伯格 / 丹妮拉·玻芭迪拉 / 素玛立·蒙塔诺 ...,动画,美国,英语,2019-03-30(美国),Nan,Nan,...,0,0,0,0,0,0,0,0,0,0
30349667,30349667,指挥家 De dirigent,玛利亚·彼特斯,克里斯蒂亚娜 ‧德‧布奥恩 / 本杰明·温赖特 / 斯科特·特纳·菲尔德 / 塞马斯·F·萨...,传记,荷兰,英语,2018-10-25(荷兰),137分钟,The Conductor,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1966460,1966460,图雅的婚事,芦苇 / 王全安,余男 / 巴特尔 / 森格,剧情 / 爱情,中国大陆,汉语普通话,2006-09-21(中国大陆),95分钟 / 86分钟(中国大陆),萨仁图雅 / Tuya's Marriage,...,0,0,0,0,0,0,0,0,0,0
3070921,3070921,饥饿 Hunger,恩达·沃尔什 / 史蒂夫·麦奎因,斯图尔特·格雷厄姆 / 莱恩·梅加 / 布赖恩·米利根 / 利亚姆·麦克马洪 / 凯伦·汉森...,剧情 / 传记,英国 / 爱尔兰,英语 / 爱尔兰语,2008-05-15(戛纳电影节) / 2008-10-31(英国/爱尔兰),96分钟,大绝食(港) / 饥饿宣言(台) / 绝食,...,0,0,0,0,0,0,0,0,0,0
1299436,1299436,风柜来的人 風櫃來的人,朱天文,钮承泽 / 张世 / 庹宗华 / 林秀玲 / 杨丽音 / 张纯芳 / 陈博正,剧情,台湾,汉语普通话 / 闽南语,1983(台湾) / 1990-07-21(日本),101分钟,The Boys From Fengkuei / All the Youthful Days,...,0,0,0,0,0,0,0,0,0,0


In [9]:
import pandas as pd
import numpy as np
import time
import os

MOVIE_DATA_PATH = 'data/m_all_info.csv'
MOVIE_DATA_COLUMNS = ['movie_id', 'title', 'director', 'scriptwriter', 'actors', 'genres',
                          'place', 'languages', 'time', 'duration', 'other_names', '_']
RATING_DATA_PATH = 'data/u_score.csv' 
RATING_DATA_CLOLUMNS = ['movie_id', 'user_id', 'rating']

ITEM_ITEM_SIMMAT_PATH = 'data/ITEM_ITEM_SIMMAT.csv'
USER_USER_SIMMAT_PATH = 'data/USER_USER_SIMMAT.csv'
PERSONA_TAG_TABLE_PATH = 'terdata/user_tag.csv'
USER_INDEX_PATH = 'terdata/u_idx.csv'

In [24]:
# 加载用户索引与id信息
users = pd.read_csv(USER_INDEX_PATH, index_col=0, header=None)
# 加载用户标签信息
tag_table = pd.read_csv(PERSONA_TAG_TABLE_PATH, header=0)
# 合并两张表
user_tag_table = pd.merge(users, tag_table, left_index=True, right_index=True)

In [25]:
user_tag_table.index = user_tag_table[1]
del(user_tag_table[1])

In [28]:
user_tag_table.T.corr()

1,165852925,0,130012755,150932792,189793987,142889559,77207438,171944917,191506217,193375266,...,163226052,142327847,178542586,59592876,168104871,140956290,188123312,173449675,137960270,102530810
1,,,,,,,,,,,,,,,,,,,,,
165852925,1.000000,0.867864,0.598936,0.458312,0.244510,0.028988,0.113630,0.780461,0.043923,0.361149,...,0.340865,0.621259,0.401338,0.066827,0.246112,0.452531,0.083809,0.882286,0.530863,0.694650
0,0.867864,1.000000,0.644944,0.465206,0.496122,0.086032,0.274721,0.785189,0.100981,0.398336,...,0.600977,0.740775,0.475120,-0.006146,0.310119,0.502743,0.398205,0.887309,0.605465,0.699235
130012755,0.598936,0.644944,1.000000,0.312841,0.652582,0.399476,0.084825,0.859000,0.153213,0.357060,...,0.490673,0.670584,0.523960,-0.051949,0.554410,0.821051,0.292343,0.742393,0.365972,0.470714
150932792,0.458312,0.465206,0.312841,1.000000,0.114622,0.120483,-0.037459,0.280238,0.354375,0.125131,...,0.467457,0.311501,0.561644,0.611052,0.146523,0.627403,0.135235,0.516447,0.147690,0.180323
189793987,0.244510,0.496122,0.652582,0.114622,1.000000,0.325411,0.622416,0.600855,0.337068,0.310688,...,0.738597,0.800713,0.596717,-0.139864,0.326355,0.492182,0.767887,0.512461,0.416325,0.406028
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140956290,0.452531,0.502743,0.821051,0.627403,0.492182,0.324812,-0.078822,0.656210,0.371155,0.428030,...,0.522918,0.524661,0.614344,0.198617,0.468046,1.000000,0.161175,0.674986,0.286965,0.293062
188123312,0.083809,0.398205,0.292343,0.135235,0.767887,0.526610,0.808901,0.262364,0.483381,0.073150,...,0.780584,0.676993,0.596404,-0.116122,0.009794,0.161175,1.000000,0.301020,0.243867,0.274966
173449675,0.882286,0.887309,0.742393,0.516447,0.512461,0.199211,0.258715,0.874834,0.275164,0.463598,...,0.626948,0.787188,0.615096,-0.016746,0.267559,0.674986,0.301020,1.000000,0.654939,0.776254


In [30]:
USER_USER_SIMMAT = pd.read_csv(USER_USER_SIMMAT_PATH, index_col=0, header=0)
USER_USER_SIMMAT

,165852925,0,130012755,150932792,189793987,142889559,77207438,171944917,191506217,193375266,...,163226052,142327847,178542586,59592876,168104871,140956290,188123312,173449675,137960270,102530810
1,,,,,,,,,,,,,,,,,,,,,
165852925,1.000000,0.867864,0.598936,0.458312,0.244510,0.028988,0.113630,0.780461,0.043923,0.361149,...,0.340865,0.621259,0.401338,0.066827,0.246112,0.452531,0.083809,0.882286,0.530863,0.694650
0,0.867864,1.000000,0.644944,0.465206,0.496122,0.086032,0.274721,0.785189,0.100981,0.398336,...,0.600977,0.740775,0.475120,-0.006146,0.310119,0.502743,0.398205,0.887309,0.605465,0.699235
130012755,0.598936,0.644944,1.000000,0.312841,0.652582,0.399476,0.084825,0.859000,0.153213,0.357060,...,0.490673,0.670584,0.523960,-0.051949,0.554410,0.821051,0.292343,0.742393,0.365972,0.470714
150932792,0.458312,0.465206,0.312841,1.000000,0.114622,0.120483,-0.037459,0.280238,0.354375,0.125131,...,0.467457,0.311501,0.561644,0.611052,0.146523,0.627403,0.135235,0.516447,0.147690,0.180323
189793987,0.244510,0.496122,0.652582,0.114622,1.000000,0.325411,0.622416,0.600855,0.337068,0.310688,...,0.738597,0.800713,0.596717,-0.139864,0.326355,0.492182,0.767887,0.512461,0.416325,0.406028
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140956290,0.452531,0.502743,0.821051,0.627403,0.492182,0.324812,-0.078822,0.656210,0.371155,0.428030,...,0.522918,0.524661,0.614344,0.198617,0.468046,1.000000,0.161175,0.674986,0.286965,0.293062
188123312,0.083809,0.398205,0.292343,0.135235,0.767887,0.526610,0.808901,0.262364,0.483381,0.073150,...,0.780584,0.676993,0.596404,-0.116122,0.009794,0.161175,1.000000,0.301020,0.243867,0.274966
173449675,0.882286,0.887309,0.742393,0.516447,0.512461,0.199211,0.258715,0.874834,0.275164,0.463598,...,0.626948,0.787188,0.615096,-0.016746,0.267559,0.674986,0.301020,1.000000,0.654939,0.776254


In [37]:
simuser_score = USER_USER_SIMMAT.loc[str(130012755)]
simuser_score = simuser_score.drop(str(130012755)) # 删掉自己
recall_users = simuser_score.sort_values(ascending=False)[:10]

In [38]:
recall_users

187714882     0.943897
142423590     0.934035
187112868     0.870413
171944917     0.859000
crastal       0.854122
148175534     0.851869
146454781     0.840286
144974534     0.821892
140956290     0.821051
WyndyHwang    0.813426
Name: 130012755, dtype: float64

In [40]:
itemA = pd.Series([30242164, 26580232, 26783513, 26788941, 27187285, 26817018, 1483507, 1309046, 26999852, 26786642, 25750923, 28086682, 1299383, 1293414, 27037053, 27060077, 1309163, 27191431, 27615441, 6860160, 26411377, 27102569, 26797690, 25716096, 27617348, 27180960, 1291557, 30267308, 26284593, 26761328, 30181455, 26776469, 1962316, 1482911, 27126359, 26633257, 27195038, 27602116, 27040737, 26928226, 1293359])
itemB = pd.Series([4896273, 30190578, 2043546, 3319755, 4217715, 1306921, 25885384, 1878865, 1945549, 1292271, 6973376, 25843976, 26628329, 26662193, 27615866, 30463396, 1966460, 1307306, 3771562, 11803087, 25964630, 26000205, 30439234, 5187955, 30254040, 27064926, 25864085, 1299247, 1297052, 1291588, 30322997, 1307394, 1300211, 27597033, 3993578])


In [42]:
list(itemA) & list(itemB)

TypeError: unsupported operand type(s) for &: 'list' and 'list'